In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import plotly.graph_objs as go 
import plotly.offline as py 
import math

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import VotingClassifier
from sklearn.base import clone 
from sklearn.metrics import ConfusionMatrixDisplay
import xgboost as xgb

In [3]:
import os

cwd = os.getcwd()
print(cwd)

d:\elliptic\elliptic++


In [4]:
import os
os.chdir("D:\\elliptic\\Elliptic_Dataset")
print("\nTransaction features: \n")
df_txs_features = pd.read_csv("txs_features.csv")
print(df_txs_features)

print("\nTransaction classes: \n")
df_txs_classes = pd.read_csv("txs_classes.csv")
print(df_txs_classes)

print("\nTransaction-Transaction edgelist: \n")
df_txs_edgelist = pd.read_csv("txs_edgelist.csv")

print(df_txs_edgelist)


Transaction features: 

             txId  Time step  Local_feature_1  Local_feature_2  \
0            3321          1        -0.169615        -0.184668   
1           11108          1        -0.137586        -0.184668   
2           51816          1        -0.170103        -0.184668   
3           68869          1        -0.114267        -0.184668   
4           89273          1         5.202107        -0.210553   
...           ...        ...              ...              ...   
203764  158304003         49        -0.165622        -0.139563   
203765  158303998         49        -0.167040        -0.139563   
203766  158303966         49        -0.167040        -0.139563   
203767  161526077         49        -0.172212        -0.139573   
203768  194103537         49        -0.172212        -0.139573   

        Local_feature_3  Local_feature_4  Local_feature_5  Local_feature_6  \
0             -1.201369        -0.121970        -0.043875        -0.113002   
1             -1.201369   

In [5]:
print("\ntxs_features.csv for txId = 272145560\n")
print(df_txs_features[df_txs_features['txId']==272145560])

print("\ntxs_classes.csv for txId = 272145560\n")
print(df_txs_classes[df_txs_classes['txId']==272145560])

print("\ntxs_edgelist.csv for txId = 272145560\n")
df_txs_edgelist[(df_txs_edgelist['txId1']==272145560) | (df_txs_edgelist['txId2']==272145560)]


txs_features.csv for txId = 272145560

             txId  Time step  Local_feature_1  Local_feature_2  \
105573  272145560         24        -0.155493        -0.107012   

        Local_feature_3  Local_feature_4  Local_feature_5  Local_feature_6  \
105573        -1.201369         -0.12197        -0.043875        -0.113002   

        Local_feature_7  Local_feature_8  ...  in_BTC_min  in_BTC_max  \
105573        -0.061584        -0.145749  ...      2.7732      2.7732   

        in_BTC_mean  in_BTC_median  in_BTC_total  out_BTC_min  out_BTC_max  \
105573       2.7732         2.7732        2.7732     0.001917     2.770883   

        out_BTC_mean  out_BTC_median  out_BTC_total  
105573        1.3864          1.3864         2.7728  

[1 rows x 184 columns]

txs_classes.csv for txId = 272145560

             txId  class
105573  272145560      1

txs_edgelist.csv for txId = 272145560



,txId1,txId2
123072,272145560,296926618
123272,272145560,272145556
125873,299475624,272145560


In [6]:
df_txs_features = df_txs_features.merge(df_txs_classes, left_on = "txId" , right_on = "txId" , how = "left")
# print(txs.shape)

print(df_txs_features["class"].value_counts())

class
3    157205
2     42019
1      4545
Name: count, dtype: int64


In [7]:
df_txs_features = df_txs_features.dropna()
df_txs_features

,txId,Time step,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,...,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total,class
0,3321,1,-0.169615,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160199,...,0.534072,0.534072,0.534072,0.534072,1.668990e-01,0.367074,0.266986,0.266986,0.533972,3
1,11108,1,-0.137586,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.127429,...,5.611878,5.611878,5.611878,5.611878,5.861940e-01,5.025584,2.805889,2.805889,5.611778,3
2,51816,1,-0.170103,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160699,...,0.456608,0.456608,0.456608,0.456608,2.279902e-01,0.228518,0.228254,0.228254,0.456508,3
3,68869,1,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,-0.161652,...,8.000000,3.102967,1.000000,9.308900,1.229000e+00,8.079800,4.654400,4.654400,9.308800,2
4,89273,1,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,5.335864,...,852.164680,852.164680,852.164680,852.164680,1.300000e-07,41.264036,0.065016,0.000441,852.164680,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202799,194747812,49,0.558398,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.584665,...,115.952889,115.952889,115.952889,115.952889,1.653300e+00,114.299544,57.976422,57.976422,115.952844,3
202800,194747925,49,0.547658,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.573676,...,114.250098,114.250098,114.250098,114.250098,2.035300e-02,114.229700,57.125027,57.125027,114.250053,3
202801,194748063,49,0.543600,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.569524,...,113.606771,113.606771,113.606771,113.606771,9.257490e-01,112.680977,56.803363,56.803363,113.606726,3
202802,194748070,49,0.537760,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.563549,...,112.680977,112.680977,112.680977,112.680977,3.026970e-01,112.378235,56.340466,56.340466,112.680932,3


In [8]:
for column in df_txs_features.columns[168:]:
    if column != "class":
        feature = np.array(df_txs_features[column]).reshape(-1,1)
        scaler = MinMaxScaler()
        scaler.fit(feature)
        feature_scaled = scaler.transform(feature)
        df_txs_features[column] = feature_scaled.reshape(1,-1)[0]

In [9]:
# remove 'unknown' transactions
data = df_txs_features["txId"] #.loc[(df_txs_features['class'] != 3), 'txId']
df_txs_features_selected = df_txs_features.loc[df_txs_features['txId'].isin(data)]
print(df_txs_features_selected["class"].value_counts())

class
3    156759
2     41500
1      4545
Name: count, dtype: int64


In [10]:
# Goal: binary classification of 0,1
# 0: licit, 1: illicit

X_data = df_txs_features_selected.loc[(df_txs_features_selected['Time step'] < 35) & (df_txs_features_selected['class'] != 3), 'txId']
X_training_timesteps = df_txs_features_selected.loc[df_txs_features_selected['txId'].isin(X_data)]
X_train = X_training_timesteps.drop(columns=['txId', 'class', 'Time step'])

X_data_test = df_txs_features_selected.loc[(df_txs_features_selected['Time step'] >= 35), 'txId']
X_testing_timesteps = df_txs_features_selected.loc[df_txs_features_selected['txId'].isin(X_data_test)]
X_test = X_testing_timesteps.drop(columns=['txId', 'class', 'Time step'])

X_data_test_class01 = df_txs_features_selected.loc[(df_txs_features_selected['Time step'] >= 35) & (df_txs_features_selected['class'] != 3), 'txId']
X_testing_timesteps_class01 = df_txs_features_selected.loc[df_txs_features_selected['txId'].isin(X_data_test_class01)]
X_test_class01 = X_testing_timesteps_class01.drop(columns=['txId', 'class', 'Time step'])

y_training_timesteps = X_training_timesteps[['class']]
y_training_timesteps = y_training_timesteps['class'].apply(lambda x: x-1 ) # change illicit (class-2) to '0' for classification
y_train = y_training_timesteps

y_testing_timesteps = X_testing_timesteps[['class']]
y_testing_timesteps = y_testing_timesteps['class'].apply(lambda x: x-1 ) # change illicit (class-2) to '0' for classification
y_test = y_testing_timesteps

y_testing_timesteps_class01 = X_testing_timesteps_class01[['class']]
y_testing_timesteps_class01 = y_testing_timesteps_class01['class'].apply(lambda x: x-1 ) # change illicit (class-2) to '0' for classification
y_test_class01 = y_testing_timesteps_class01

In [11]:
y_train.value_counts()

class
1    26237
0     3462
Name: count, dtype: int64

In [12]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_SMOTE, y_train_SMOTE = smote.fit_resample(X_train, y_train)


In [15]:
y_train_SMOTE.value_counts()

class
1    26237
0    26237
Name: count, dtype: int64

In [13]:
# RANDOM FOREST (RF)
cRF = RandomForestClassifier(criterion='entropy', n_estimators=50, max_depth=32 ,class_weight='balanced').fit(X_train.values,y_train.values)
y_preds_RF_class01 = cRF.predict(X_test_class01.values)
print(y_test_class01.shape)
prec,rec,f1,num = precision_recall_fscore_support(y_test_class01.values, y_preds_RF_class01)


print("RANDOM FOREST (RF)\n")


# Đánh giá từng class
print("Detailed per-class evaluation:")
for i, label in enumerate([0, 1]):
    print(f"Class {label}:")
    print(f"  Precision: {prec[i]:.3f}")
    print(f"  Recall:    {rec[i]:.3f}")
    print(f"  F1 Score:  {f1[i]:.3f}")
    print(f"  Support:   {num[i]}")
    print()

# Ma trận nhầm lẫn
cm = confusion_matrix(y_test_class01.values, y_preds_RF_class01, labels=[0, 1])
print("Confusion Matrix:")
print(cm)
print()

# Đánh giá chung
print("Overall Evaluation:\n")
# Micro F1
micro_f1 = f1_score(y_test_class01.values, y_preds_RF_class01, average='micro')
print("Micro-Average F1 Score: %.3f\n" % micro_f1)

# Macro-average F1
macro_f1 = f1_score(y_test_class01.values, y_preds_RF_class01, average='macro')
print(f"Macro-Average F1 Score: {macro_f1:.3f}")

# Weighted-average F1
weighted_f1 = f1_score(y_test_class01.values, y_preds_RF_class01, average='weighted')
print(f"Weighted-Average F1 Score: {weighted_f1:.3f}")

# Accuracy from confusion matrix
accuracy = np.sum(np.diag(cm)) / np.sum(cm)
print(f"Overall Accuracy: {accuracy:.3f}")

# Interpretation of confusion matrix
print("\nConfusion Matrix Analysis:")
total_samples = np.sum(cm)
for i, label in enumerate([0, 1]):
    true_positives = cm[i, i]
    false_positives = np.sum(cm[:, i]) - true_positives
    false_negatives = np.sum(cm[i, :]) - true_positives
    print(f"Class {label}:")
    print(f"  True Positives: {true_positives} ({(true_positives / num[i] * 100):.1f}% of class {label} samples)")
    print(f"  False Positives: {false_positives}")
    print(f"  False Negatives: {false_negatives}")
    print()

(16346,)
RANDOM FOREST (RF)

Detailed per-class evaluation:
Class 0:
  Precision: 0.981
  Recall:    0.715
  F1 Score:  0.827
  Support:   1083

Class 1:
  Precision: 0.980
  Recall:    0.999
  F1 Score:  0.989
  Support:   15263

Confusion Matrix:
[[  774   309]
 [   15 15248]]

Overall Evaluation:

Micro-Average F1 Score: 0.980

Macro-Average F1 Score: 0.908
Weighted-Average F1 Score: 0.979
Overall Accuracy: 0.980

Confusion Matrix Analysis:
Class 0:
  True Positives: 774 (71.5% of class 0 samples)
  False Positives: 15
  False Negatives: 309

Class 1:
  True Positives: 15248 (99.9% of class 1 samples)
  False Positives: 309
  False Negatives: 15



In [14]:
# RANDOM FOREST (RF) + SMOTE
cRF = RandomForestClassifier(criterion='gini', n_estimators=50, max_depth=32 ,class_weight='balanced').fit(X_train_SMOTE.values,y_train_SMOTE.values)
y_preds_RF_class01 = cRF.predict(X_test_class01.values)
print(y_test_class01.shape)
prec,rec,f1,num = precision_recall_fscore_support(y_test_class01.values, y_preds_RF_class01)


print("RANDOM FOREST (RF) + SMOTE\n")


# Đánh giá từng class
print("Detailed per-class evaluation:")
for i, label in enumerate([0, 1]):
    print(f"Class {label}:")
    print(f"  Precision: {prec[i]:.3f}")
    print(f"  Recall:    {rec[i]:.3f}")
    print(f"  F1 Score:  {f1[i]:.3f}")
    print(f"  Support:   {num[i]}")
    print()

# Ma trận nhầm lẫn
cm = confusion_matrix(y_test_class01.values, y_preds_RF_class01, labels=[0, 1])
print("Confusion Matrix:")
print(cm)
print()

# Đánh giá chung
print("Overall Evaluation:\n")
# Micro F1
micro_f1 = f1_score(y_test_class01.values, y_preds_RF_class01, average='micro')
print("Micro-Average F1 Score: %.3f\n" % micro_f1)

# Macro-average F1
macro_f1 = f1_score(y_test_class01.values, y_preds_RF_class01, average='macro')
print(f"Macro-Average F1 Score: {macro_f1:.3f}")

# Weighted-average F1
weighted_f1 = f1_score(y_test_class01.values, y_preds_RF_class01, average='weighted')
print(f"Weighted-Average F1 Score: {weighted_f1:.3f}")

# Accuracy from confusion matrix
accuracy = np.sum(np.diag(cm)) / np.sum(cm)
print(f"Overall Accuracy: {accuracy:.3f}")

# Interpretation of confusion matrix
print("\nConfusion Matrix Analysis:")
total_samples = np.sum(cm)
for i, label in enumerate([0, 1]):
    true_positives = cm[i, i]
    false_positives = np.sum(cm[:, i]) - true_positives
    false_negatives = np.sum(cm[i, :]) - true_positives
    print(f"Class {label}:")
    print(f"  True Positives: {true_positives} ({(true_positives / num[i] * 100):.1f}% of class {label} samples)")
    print(f"  False Positives: {false_positives}")
    print(f"  False Negatives: {false_negatives}")
    print()

(16346,)
RANDOM FOREST (RF) + SMOTE

Detailed per-class evaluation:
Class 0:
  Precision: 0.925
  Recall:    0.718
  F1 Score:  0.809
  Support:   1083

Class 1:
  Precision: 0.980
  Recall:    0.996
  F1 Score:  0.988
  Support:   15263

Confusion Matrix:
[[  778   305]
 [   63 15200]]

Overall Evaluation:

Micro-Average F1 Score: 0.977

Macro-Average F1 Score: 0.898
Weighted-Average F1 Score: 0.976
Overall Accuracy: 0.977

Confusion Matrix Analysis:
Class 0:
  True Positives: 778 (71.8% of class 0 samples)
  False Positives: 63
  False Negatives: 305

Class 1:
  True Positives: 15200 (99.6% of class 1 samples)
  False Positives: 305
  False Negatives: 63



In [16]:
from lightgbm import LGBMClassifier

cLGB = LGBMClassifier(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=32,
    random_state=42,
    class_weight='balanced'
).fit(X_train.values,y_train.values)
y_preds_LGB_class01 = cLGB.predict(X_test_class01.values)
prec,rec,f1,num = precision_recall_fscore_support(y_test_class01.values, y_preds_LGB_class01)

print("LightGBM (LGB)\n")


# Đánh giá từng class
print("Detailed per-class evaluation:")
for i, label in enumerate([0, 1]):
    print(f"Class {label}:")
    print(f"  Precision: {prec[i]:.3f}")
    print(f"  Recall:    {rec[i]:.3f}")
    print(f"  F1 Score:  {f1[i]:.3f}")
    print(f"  Support:   {num[i]}")
    print()

# Ma trận nhầm lẫn
cm = confusion_matrix(y_test_class01.values, y_preds_LGB_class01, labels=[0, 1])
print("Confusion Matrix:")
print(cm)
print()

# Đánh giá chung
print("Overall Evaluation:\n")
# Micro F1
micro_f1 = f1_score(y_test_class01.values, y_preds_LGB_class01, average='micro')
print("Micro-Average F1 Score: %.3f\n" % micro_f1)

# Macro-average F1
macro_f1 = f1_score(y_test_class01.values, y_preds_LGB_class01, average='macro')
print(f"Macro-Average F1 Score: {macro_f1:.3f}")

# Weighted-average F1
weighted_f1 = f1_score(y_test_class01.values, y_preds_LGB_class01, average='weighted')
print(f"Weighted-Average F1 Score: {weighted_f1:.3f}")

# Accuracy from confusion matrix
accuracy = np.sum(np.diag(cm)) / np.sum(cm)
print(f"Overall Accuracy: {accuracy:.3f}")

# Interpretation of confusion matrix
print("\nConfusion Matrix Analysis:")
total_samples = np.sum(cm)
for i, label in enumerate([0, 1]):
    true_positives = cm[i, i]
    false_positives = np.sum(cm[:, i]) - true_positives
    false_negatives = np.sum(cm[i, :]) - true_positives
    print(f"Class {label}:")
    print(f"  True Positives: {true_positives} ({(true_positives / num[i] * 100):.1f}% of class {label} samples)")
    print(f"  False Positives: {false_positives}")
    print(f"  False Negatives: {false_negatives}")
    print()

[LightGBM] [Info] Number of positive: 26237, number of negative: 3462
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014480 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 41994
[LightGBM] [Info] Number of data points in the train set: 29699, number of used features: 181
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
LightGBM (LGB)

Detailed per-class evaluation:
Class 0:
  Precision: 0.893
  Recall:    0.734
  F1 Score:  0.806
  Support:   1083

Class 1:
  Precision: 0.981
  Recall:    0.994
  F1 Score:  0.988
  Support:   15263

Confusion Matrix:
[[  795   288]
 [   95 15168]]

Overall Evaluation:

Micro-Average F1 Score: 0.977

Macro-Average F1 Score: 0.897
Weighted-Average F1 Score: 0.975
Overall Accuracy: 0.977

Confusion Matrix Analysis:
Class 0:
  True Positives: 795 (73.4% of class 0 samples)
  False Positi

In [17]:
# LGBM + SMOTE
from lightgbm import LGBMClassifier

cLGB = LGBMClassifier(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=32,
    random_state=42,
    class_weight='balanced'
).fit(X_train_SMOTE.values,y_train_SMOTE.values)
y_preds_LGB_class01 = cLGB.predict(X_test_class01.values)
prec,rec,f1,num = precision_recall_fscore_support(y_test_class01.values, y_preds_LGB_class01)

print("LightGBM (LGB)\n")


# Đánh giá từng class
print("Detailed per-class evaluation:")
for i, label in enumerate([0, 1]):
    print(f"Class {label}:")
    print(f"  Precision: {prec[i]:.3f}")
    print(f"  Recall:    {rec[i]:.3f}")
    print(f"  F1 Score:  {f1[i]:.3f}")
    print(f"  Support:   {num[i]}")
    print()

# Ma trận nhầm lẫn
cm = confusion_matrix(y_test_class01.values, y_preds_LGB_class01, labels=[0, 1])
print("Confusion Matrix:")
print(cm)
print()

# Đánh giá chung
print("Overall Evaluation:\n")
# Micro F1
micro_f1 = f1_score(y_test_class01.values, y_preds_LGB_class01, average='micro')
print("Micro-Average F1 Score: %.3f\n" % micro_f1)

# Macro-average F1
macro_f1 = f1_score(y_test_class01.values, y_preds_LGB_class01, average='macro')
print(f"Macro-Average F1 Score: {macro_f1:.3f}")

# Weighted-average F1
weighted_f1 = f1_score(y_test_class01.values, y_preds_LGB_class01, average='weighted')
print(f"Weighted-Average F1 Score: {weighted_f1:.3f}")

# Accuracy from confusion matrix
accuracy = np.sum(np.diag(cm)) / np.sum(cm)
print(f"Overall Accuracy: {accuracy:.3f}")

# Interpretation of confusion matrix
print("\nConfusion Matrix Analysis:")
total_samples = np.sum(cm)
for i, label in enumerate([0, 1]):
    true_positives = cm[i, i]
    false_positives = np.sum(cm[:, i]) - true_positives
    false_negatives = np.sum(cm[i, :]) - true_positives
    print(f"Class {label}:")
    print(f"  True Positives: {true_positives} ({(true_positives / num[i] * 100):.1f}% of class {label} samples)")
    print(f"  False Positives: {false_positives}")
    print(f"  False Negatives: {false_negatives}")
    print()

[LightGBM] [Info] Number of positive: 26237, number of negative: 26237
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017252 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 45407
[LightGBM] [Info] Number of data points in the train set: 52474, number of used features: 181
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
LightGBM (LGB)

Detailed per-class evaluation:
Class 0:
  Precision: 0.929
  Recall:    0.733
  F1 Score:  0.819
  Support:   1083

Class 1:
  Precision: 0.981
  Recall:    0.996
  F1 Score:  0.989
  Support:   15263

Confusion Matrix:
[[  794   289]
 [   61 15202]]

Overall Evaluation:

Micro-Average F1 Score: 0.979

Macro-Average F1 Score: 0.904
Weighted-Average F1 Score: 0.977
Overall Accuracy: 0.979

Confusion Matrix Analysis:
Class 0:
  True Positives: 794 (73.3% of class 0 samples)
  False Positives: 61
  False Negatives: 289

Class 1:
  True Posi

In [20]:
# XGBOOST (XGB)
cXGB = xgb.XGBClassifier(objective="multi:softmax", num_class=2, random_state=42, max_depth=32)
cXGB.fit(X_train.values, y_train.values)
y_preds_XGB_class01 = cXGB.predict(X_test_class01.values)
prec,rec,f1,num = precision_recall_fscore_support(y_test_class01.values, y_preds_XGB_class01)

print("XGBOOST + SMOTE")
print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f"%(prec[0],rec[0],f1[0]))
micro_f1 = f1_score(y_test_class01, y_preds_XGB_class01, average='micro')

print("Micro-Average F1 Score: %.3f"%(micro_f1))
macro_f1 = f1_score(y_test_class01.values, y_preds_LGB_class01, average='macro')
print(f"Macro-Average F1 Score: {macro_f1:.3f}")
#print(confusion_matrix(y, y_pred))
cm = confusion_matrix(y_test_class01.values, y_preds_XGB_class01, labels=[0, 1])
print(cm)

XGBOOST + SMOTE
Precision: 0.921 
Recall: 0.726 
F1 Score: 0.812
Micro-Average F1 Score: 0.978
Macro-Average F1 Score: 0.904
[[  786   297]
 [   67 15196]]


In [19]:
# XGBOOST (XGB) + SMOTE
cXGB = xgb.XGBClassifier(objective="multi:softmax", num_class=2, random_state=42, max_depth=32)
cXGB.fit(X_train_SMOTE.values, y_train_SMOTE.values)
y_preds_XGB_class01 = cXGB.predict(X_test_class01.values)
prec,rec,f1,num = precision_recall_fscore_support(y_test_class01.values, y_preds_XGB_class01)

print("XGBOOST + SMOTE")
print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f"%(prec[0],rec[0],f1[0]))
micro_f1 = f1_score(y_test_class01, y_preds_XGB_class01, average='micro')
print("Micro-Average F1 Score: %.3f"%(micro_f1))
#print(confusion_matrix(y, y_pred))
cm = confusion_matrix(y_test_class01.values, y_preds_XGB_class01, labels=[0, 1])
print(cm)

XGBOOST + SMOTE
Precision: 0.908 
Recall: 0.728 
F1 Score: 0.808
Micro-Average F1 Score: 0.977
[[  788   295]
 [   80 15183]]


In [ ]:
#create a dictionary of our models
estimatorsLGBRF=[('RF', cRF), ('LGB', cLGB)]
#create our voting classifier, inputting our models
ensembleLGBRF = VotingClassifier(estimatorsLGBRF, voting='soft')
ensembleLGBRF.fit(X_train.values, y_train.values)
y_preds_LGBRF_class01 = ensembleLGBRF.predict(X_test_class01.values)
prec,rec,f1,num = precision_recall_fscore_support(y_test_class01.values, y_preds_LGBRF_class01)

print("Ensemble: LightGBM (LGB) + Random Forest (RF)\n")

# In kết quả chi tiết cho từng lớp
for i, label in enumerate([0, 1]):
    print(f"Class {label}:")
    print(f"  Precision: {prec[i]:.3f}")
    print(f"  Recall:    {rec[i]:.3f}")
    print(f"  F1 Score:  {f1[i]:.3f}")
    print(f"  Support:   {num[i]}")
    print()

# Micro F1
micro_f1 = f1_score(y_test_class01.values, y_preds_LGBRF_class01, average='micro')
print("Micro-Average F1 Score: %.3f\n" % micro_f1)

# Confusion Matrix
cm = confusion_matrix(y_test_class01.values, y_preds_LGBRF_class01, labels=[0, 1])
print("Confusion Matrix:")
print(cm)
print()

# Overall Evaluation
print("Overall Evaluation:\n")
# Macro-average F1
macro_f1 = f1_score(y_test_class01.values, y_preds_LGBRF_class01, average='macro')
print(f"Macro-Average F1 Score: {macro_f1:.3f}")

# Weighted-average F1
weighted_f1 = f1_score(y_test_class01.values, y_preds_LGBRF_class01, average='weighted')
print(f"Weighted-Average F1 Score: {weighted_f1:.3f}")

# Overall Accuracy
accuracy = np.sum(np.diag(cm)) / np.sum(cm)
print(f"Overall Accuracy: {accuracy:.3f}")

# Confusion Matrix Analysis
print("\nConfusion Matrix Analysis:")
for i, label in enumerate([0, 1]):
    true_positives = cm[i, i]
    false_positives = np.sum(cm[:, i]) - true_positives
    false_negatives = np.sum(cm[i, :]) - true_positives
    print(f"Class {label}:")
    print(f"  True Positives: {true_positives} ({(true_positives / num[i] * 100):.1f}% of class {label} samples)")
    print(f"  False Positives: {false_positives}")
    print(f"  False Negatives: {false_negatives}")
    print()

[LightGBM] [Info] Number of positive: 26237, number of negative: 3462
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010163 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 41994
[LightGBM] [Info] Number of data points in the train set: 29699, number of used features: 181
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
Ensemble: LightGBM (LGB) + Random Forest (RF)

Class 0:
  Precision: 0.930
  Recall:    0.727
  F1 Score:  0.816
  Support:   1083

Class 1:
  Precision: 0.981
  Recall:    0.996
  F1 Score:  0.988
  Support:   15263

Micro-Average F1 Score: 0.978

Confusion Matrix:
[[  787   296]
 [   59 15204]]

Overall Evaluation:

Macro-Average F1 Score: 0.902
Weighted-Average F1 Score: 0.977
Overall Accuracy: 0.978

Confusion Matrix Analysis:
Class 0:
  True Positives: 787 (72.7% of class 0 samples)
  False Positi

In [ ]:
#create a dictionary of our models
estimatorsLGBRF=[('RF', cRF), ('LGB', cLGB)]
#create our voting classifier, inputting our models
ensembleLGBRF = VotingClassifier(estimatorsLGBRF, voting='soft')
ensembleLGBRF.fit(X_train_SMOTE.values, y_train_SMOTE.values)
y_preds_LGBRF_class01 = ensembleLGBRF.predict(X_test_class01.values)
prec,rec,f1,num = precision_recall_fscore_support(y_test_class01.values, y_preds_LGBRF_class01)

print("Ensemble: LightGBM (LGB) + Random Forest (RF) + SMOTE\n")

# In kết quả chi tiết cho từng lớp
for i, label in enumerate([0, 1]):
    print(f"Class {label}:")
    print(f"  Precision: {prec[i]:.3f}")
    print(f"  Recall:    {rec[i]:.3f}")
    print(f"  F1 Score:  {f1[i]:.3f}")
    print(f"  Support:   {num[i]}")
    print()

# Micro F1
micro_f1 = f1_score(y_test_class01.values, y_preds_LGBRF_class01, average='micro')
print("Micro-Average F1 Score: %.3f\n" % micro_f1)

# Confusion Matrix
cm = confusion_matrix(y_test_class01.values, y_preds_LGBRF_class01, labels=[0, 1])
print("Confusion Matrix:")
print(cm)
print()

# Overall Evaluation
print("Overall Evaluation:\n")
# Macro-average F1
macro_f1 = f1_score(y_test_class01.values, y_preds_LGBRF_class01, average='macro')
print(f"Macro-Average F1 Score: {macro_f1:.3f}")

# Weighted-average F1
weighted_f1 = f1_score(y_test_class01.values, y_preds_LGBRF_class01, average='weighted')
print(f"Weighted-Average F1 Score: {weighted_f1:.3f}")

# Overall Accuracy
accuracy = np.sum(np.diag(cm)) / np.sum(cm)
print(f"Overall Accuracy: {accuracy:.3f}")

# Confusion Matrix Analysis
print("\nConfusion Matrix Analysis:")
for i, label in enumerate([0, 1]):
    true_positives = cm[i, i]
    false_positives = np.sum(cm[:, i]) - true_positives
    false_negatives = np.sum(cm[i, :]) - true_positives
    print(f"Class {label}:")
    print(f"  True Positives: {true_positives} ({(true_positives / num[i] * 100):.1f}% of class {label} samples)")
    print(f"  False Positives: {false_positives}")
    print(f"  False Negatives: {false_negatives}")
    print()

[LightGBM] [Info] Number of positive: 26237, number of negative: 26237
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016046 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 45407
[LightGBM] [Info] Number of data points in the train set: 52474, number of used features: 181
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Ensemble: LightGBM (LGB) + Random Forest (RF) + SMOTE

Class 0:
  Precision: 0.946
  Recall:    0.730
  F1 Score:  0.824
  Support:   1083

Class 1:
  Precision: 0.981
  Recall:    0.997
  F1 Score:  0.989
  Support:   15263

Micro-Average F1 Score: 0.979

Confusion Matrix:
[[  791   292]
 [   45 15218]]

Overall Evaluation:

Macro-Average F1 Score: 0.907
Weighted-Average F1 Score: 0.978
Overall Accuracy: 0.979

Confusion Matrix Analysis:
Class 0:
  True Positives: 791 (73.0% of class 0 samples)
  False Positives: 45
  False Negatives: 292

Class 1:
  T